In [1]:
import prior
dataset = prior.load_dataset("procthor-10k")

house = dataset["train"][20]

AE: https://api.github.com/repos/allenai/procthor-10k/commits?sha=main
[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 9223.36it/s]


In [3]:
print(house)
rooms = []
for room in house["rooms"]:
    room_poly = [(corner["x"], corner["z"]) for corner in room["floorPolygon"]]
    #print(room["roomType"] + " # " + str(room["floorPolygon"]))
    #print(room["roomType"] + " ?? " + str(room_poly))
    rooms.append((room["roomType"], room_poly))

print(rooms)

for room in rooms:
    print(room[1])

{'doors': [{'assetId': 'Doorway_Double_7', 'id': 'door|4|5', 'openable': True, 'openness': 1, 'room0': 'room|5', 'room1': 'room|4', 'wall0': 'wall|5|8.35|0.00|8.35|6.68', 'wall1': 'wall|4|8.35|0.00|8.35|6.68', 'holePolygon': [{'x': 4.353640555830781, 'y': 0, 'z': 0}, {'x': 6.301061391325776, 'y': 2.0971285314299166, 'z': 0}], 'assetPosition': {'x': 5.32681868454266, 'y': 1.0462727337144315, 'z': 0}}, {'assetId': 'Doorway_5', 'id': 'door|6|7', 'openable': True, 'openness': 1, 'room0': 'room|7', 'room1': 'room|6', 'wall0': 'wall|7|6.68|10.02|6.68|11.69', 'wall1': 'wall|6|6.68|10.02|6.68|11.69', 'holePolygon': [{'x': 0.5769642321808005, 'y': 0, 'z': 0}, {'x': 1.6145027606231834, 'y': 2.0979503514245152, 'z': 0}], 'assetPosition': {'x': 1.0944141168562438, 'y': 1.0459483740851283, 'z': 0}}, {'assetId': 'Doorway_5', 'id': 'door|4|6', 'openable': True, 'openness': 1, 'room0': 'room|4', 'room1': 'room|6', 'wall0': 'wall|4|8.35|6.68|13.36|6.68', 'wall1': 'wall|6|8.35|6.68|13.36|6.68', 'holePol

In [78]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

def is_point_inside_room(point_to_test, room_polygon):
    (x, y, z) = point_to_test
    point = Point(x, z)
    polygon = Polygon(room_polygon)
    return polygon.contains(point)

def what_room_is_point_in(rooms, point):
    for room in rooms:
        if is_point_inside_room(point, room[1]):
            return room[0]
    return "NONE"

True


In [8]:
import glob
import pickle

def process_scene_files():
    #pkl_mistral = "pkl_mistral/diff*.pkl"
    pkl_gemma = "pkl_gemma/scene_descr_train_*.pkl"
    #print(glob.glob(pkl_gemma)) 
    
    gemma_scene_files = glob.glob(pkl_gemma) # files showing gemma scenes
    
    for gemma_scene_f in gemma_scene_files:
        f = open(gemma_scene_f,'rb')
        gemma_scene = pickle.load(f)
        print(gemma_scene_f)

        room_points = gemma_scene.get_all_points()


#    new_point = {
#        "point_pose" : point_pose,
#        "room_type_llm" : room_type_llm,
#        "room_type_svc" : room_type_svc,
#        "room_type_gt" : room_type_gt,
#        "visible_objects_at_this_point" : visible_objects_at_this_point,
#        "visible_object_names" : set(),
#        "elapsed_time_llm": elapsed_time_llm,
#        "elapsed_time_svc": elapsed_time_svc
#    }
        gt_vs_llm = 0
        gt_vs_svc = 0
        llm_vs_svc = 0
        
        for rp in room_points:
            #pp = rp["point_pose"]
            #(pos, rtn) = rp["point_pose"]
            #print(rp["room_type_llm"].name + " # " + rp["room_type_svc"].name + " # " + rp["room_type_gt"].name + " # " + str(rp["elapsed_time_llm"]) + " # " + str(rp["elapsed_time_svc"]))
            if rp["room_type_gt"] != rp["room_type_llm"]:
                gt_vs_llm += 1
                print(rp["room_type_llm"].name + " # " + rp["room_type_svc"].name + " # " + rp["room_type_gt"].name + " # " + str(rp["elapsed_time_llm"]) + " # " + str(rp["elapsed_time_svc"]))
            if rp["room_type_gt"] != rp["room_type_svc"]:
                gt_vs_svc += 1
            if rp["room_type_llm"] != rp["room_type_svc"]:
                llm_vs_svc += 1
        
        print("gt_vs_llm = " + str(gt_vs_llm) + " # " + "gt_vs_svc = " + str(gt_vs_svc) + " # " + "llm_vs_svc = " + str(llm_vs_svc) + " # pc = " + str(len(room_points)))
        
        f.close()
        print("\n")

#scene_descr_llm_fname = "pkl_mistral/scene_descriptions_llm.pkl"
#scene_descr_svc_fname = "pkl_mistral/scene_descriptions_svc.pkl"
#if (os.path.isfile(scene_descr_llm_fname) and os.path.isfile(scene_descr_svc_fname)):
#    file_llm = open(scene_descr_llm_fname,'rb')
#    file_svc = open(scene_descr_svc_fname,'rb')
#    scene_descriptions_llm = pickle.load(file_llm)

process_scene_files()

pkl_gemma/scene_descr_train_1.pkl
BEDROOM # BEDROOM # BEDROOM # 29.03196 # 0.0063
LIVING_ROOM # BEDROOM # BEDROOM # 15.52045 # 0.00077
KITCHEN # LIVING_ROOM # BEDROOM # 12.90399 # 0.00088
KITCHEN # LIVING_ROOM # BEDROOM # 14.88548 # 0.00087
LIVING_ROOM # LIVING_ROOM # LIVING_ROOM # 14.65532 # 0.00082
LIVING_ROOM # LIVING_ROOM # LIVING_ROOM # 18.59854 # 0.00099
LIVING_ROOM # LIVING_ROOM # BEDROOM # 17.44463 # 0.00082
BEDROOM # BEDROOM # BEDROOM # 13.80724 # 0.00083
LIVING_ROOM # BEDROOM # BEDROOM # 20.4077 # 0.00122
LIVING_ROOM # BEDROOM # BEDROOM # 17.74193 # 0.00083
LIVING_ROOM # LIVING_ROOM # BEDROOM # 27.29662 # 0.00085
BEDROOM # BEDROOM # BEDROOM # 13.17631 # 0.00081
KITCHEN # BATHROOM # BATHROOM # 18.91694 # 0.00087
BATHROOM # BATHROOM # BATHROOM # 11.1252 # 0.00091
BATHROOM # BATHROOM # BATHROOM # 21.70794 # 0.00083
KITCHEN # BATHROOM # BATHROOM # 16.71876 # 0.00085
LIVING_ROOM # LIVING_ROOM # BEDROOM # 16.92066 # 0.00086
BEDROOM # BEDROOM # BEDROOM # 22.25731 # 0.00088
LIVING_RO